In [1]:

import pandas as pd
import numpy as np
import datetime
from sklearn.ensemble import RandomForestClassifier
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn.naive_bayes import GaussianNB
#import xgboost
#from xgboost import XGBClassifier
from sklearn.preprocessing import normalize
from sklearn import metrics
from ggplot import *
import matplotlib.pyplot as plt
import time

In [2]:
#loads the datasets
DF_fxrates = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/fxrates.csv")
DF_equity = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/equity.csv")
DF_10yrbond = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/10yrbond.csv")
DF_fundreturns = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/fundreturns.csv")
DF_benchmarks = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/benchmarks.csv")
DF_12monthrates = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/12monthrates.csv")



In [3]:

#convert to pandas
DF_fxrates = DF_fxrates.toPandas()
DF_equity = DF_equity.toPandas()
DF_10yrbond = DF_10yrbond.toPandas()
DF_fundreturns = DF_fundreturns.toPandas()
DF_benchmarks = DF_benchmarks.toPandas()
DF_12monthrates = DF_12monthrates.toPandas()

In [4]:
#selects the fund analysed and its benchmark
DF_fund_analysis = DF_fundreturns[['Fund Name','Global Select Fund']]
DF_benchmark_analysis = DF_benchmarks[['Period','M990100.N']]
DF_fund_analysis = DF_fund_analysis.rename(columns={'Fund Name': 'Period'}) 

In [5]:
#merges the two
DF_fund_analysis = DF_fund_analysis.merge(DF_benchmark_analysis, on='Period', how='inner')

#computing outperfomance
DF_fund_analysis["outperfomance"] = DF_fund_analysis[DF_fund_analysis.columns[1]].astype(float) - DF_fund_analysis[DF_fund_analysis.columns[2]].astype(float)

In [6]:
#create lags

DF_fund_analysis = pd.concat([DF_fund_analysis, DF_fund_analysis["outperfomance"].shift(), DF_fund_analysis["outperfomance"].shift(2), DF_fund_analysis["outperfomance"].shift(3), DF_fund_analysis["outperfomance"].shift(4)], axis=1)

DF_fund_analysis.columns = ['Period','Fund','Benchmark', 'outperfomance', 'outperfomanceL1', 'outperfomanceL2', 'outperfomanceL3', 'outperfomanceL4']



In [7]:
#data transformations
DF_fund_analysis["Fund_outperforms"] = np.where(DF_fund_analysis["outperfomance"]>0, 1, 0)

#tidying up the dataset
DF_fund_analysis = DF_fund_analysis.dropna()

In [8]:

# Split dataset into train and test
DF_fund_analysis['Period'] = [datetime.datetime.strptime(date,'%d/%m/%Y') for date in DF_fund_analysis['Period']]
train_df = DF_fund_analysis[DF_fund_analysis["Period"]<='2011-01-15']
train_df = train_df.reset_index()
test_df = DF_fund_analysis[DF_fund_analysis["Period"]>'2011-01-15']
test_df = test_df.reset_index()

In [9]:
# Separate the X & Y
X_cols = ['outperfomanceL1', 'outperfomanceL2', 'outperfomanceL3',
       'outperfomanceL4']
Y_cols = ['Fund_outperforms']
X_train = train_df[X_cols]
Y_train = train_df[Y_cols]
X_test = test_df[X_cols]
Y_test = test_df[Y_cols]


In [10]:
######################################################
# Random Forest Model
######################################################
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(X_train, Y_train["Fund_outperforms"])
Y_pred = forest.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)

In [11]:
importances = forest.feature_importances_
importances

Out[ 94 ]: array([ 0.27157629, 0.24596242, 0.23466023, 0.24780106])

In [12]:
test_df["model_prediction"] = Y_pred
test_df["model_return"] = np.where(test_df["model_prediction"] == 1, test_df["Fund"], test_df["Benchmark"])
test_df.head()

Out[ 19 ]: 
 index Period Fund Benchmark outperfomance outperfomanceL1 \
0 338 2011-01-31 -0.021229102 -0.000455 -0.020774 0.014838 
1 339 2011-02-28 -0.004276144 0.019263 -0.023539 -0.020774 
2 340 2011-03-31 0.024359403 0.004685 0.019674 -0.023539 
3 341 2011-04-30 0.005193898 0.001857 0.003337 0.019674 
4 342 2011-05-31 0.000834144 -0.007657 0.008491 0.003337 

 outperfomanceL2 outperfomanceL3 outperfomanceL4 Fund_outperforms \
0 -0.008438 0.000262 0.009278 0 
1 0.014838 -0.008438 0.000262 0 
2 -0.020774 0.014838 -0.008438 1 
3 -0.023539 -0.020774 0.014838 1 
4 0.019674 -0.023539 -0.020774 1 

 model_prediction model_return 
0 1 -0.021229102 
1 1 -0.004276144 
2 0 0.00468499 
3 0 0.00185694 
4 0 -0.00765654

In [13]:
test_df["fund_cumulative_return"] = (1 + test_df["Fund"].astype(float)) 
test_df["Benchmark_cumulative_return"] = (1 + test_df["Benchmark"].astype(float))  
test_df["Model_cumulative_return"] = (1 + test_df["model_return"].astype(float))  

for months in range(1,test_df.shape[0]):
  test_df["fund_cumulative_return"][months] = (1 + test_df["Fund"].astype(float)[months]) * test_df["fund_cumulative_return"][months-1]
  test_df["Benchmark_cumulative_return"][months] = (1 + test_df["Benchmark"].astype(float)[months]) * test_df["Benchmark_cumulative_return"][months-1]
  test_df["Model_cumulative_return"][months] = (1 + test_df["model_return"].astype(float)[months]) * test_df["Model_cumulative_return"][months-1] 
  
  print (test_df["Fund"][months])

/local_disk0/tmp/1548867539297-0/PythonShell.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import os
/local_disk0/tmp/1548867539297-0/PythonShell.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import pickle
/local_disk0/tmp/1548867539297-0/PythonShell.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import signal
-0.004276144
0.024359403
0.005193898
0.000834144
0.00544057
-0.050265951
-0.068249042
-0.062007234
0.083166889
-0.060774471
0.042565375
0.033922687
0.069489768
-0.006126118
-0.031152574
-0.052959961
0.007314416
0.029354207
0.018160897
0.012603803
0.003105665
0.025590789
0.022405132
0.085995709
0.04282164
0.013708117
-0.010428388
0.035878901
-0.035243845
0.03553578
-0.03131438
0.003038221
0.03923588
-0.012805068
0.000413345
-0.039133514
0.022175809
-0.001141827
-0.002045605
0.024075079
-0.021075767
-0.0030871
0.013009994
0.00289809
0.013914179
0.044058797
-0.002355845
0.017391956
0.026581439
0.043298821
-0.022290255
0.020581923
-0.049900028
0.03198418
-0.054338654
-0.019853884
0.05179706
0.012929169
-0.003720649
-0.043456508
0.039210192
0.026226385
-0.006831319
0.004867725
0.053127633
0.078862724
0.010782421
-0.011431541
0.05316534
0.016484646
0.023683336
0.004316729
0.03313528
-0.001223624
-0.013598867
0.027020686
0.002297739
0.001635564
0.000669219
-0.015475362
0.046354287
-0.001272413
0.029120786
-0.005962532
-0.028301287
-0.059297429
0.057654877
0.031153467
0.013015489
0.031308987
0.023598604
-0.011169655
-0.054441779
0.035558105
-0.099465675

In [14]:
from pyspark import SparkContext
test_df["model_return"] = test_df["model_return"].astype(float)
spark_df = spark.createDataFrame(test_df)
display(spark_df)

index,Period,Fund,Benchmark,outperfomance,outperfomanceL1,outperfomanceL2,outperfomanceL3,outperfomanceL4,Fund_outperforms,model_prediction,model_return,fund_cumulative_return,Benchmark_cumulative_return,Model_cumulative_return
338,2011-01-31T00:00:00.000+0000,-0.021229102,-4.55172E-4,-0.02077393,0.014838050000000005,-0.008438306999999999,2.619759999999971E-4,0.009277641000000003,0,1,-0.021229102,0.9787708980000001,0.999544828,0.9787708980000001
339,2011-02-28T00:00:00.000+0000,-0.004276144,0.019263272,-0.023539416,-0.02077393,0.014838050000000005,-0.008438306999999999,2.619759999999971E-4,0,1,-0.004276144,0.9745855326971428,1.0187993318979573,0.9745855326971428
340,2011-03-31T00:00:00.000+0000,0.024359403,0.004684993,0.019674410000000003,-0.023539416,-0.02077393,0.014838050000000005,-0.008438306999999999,1,0,0.004684993,0.9983258544460822,1.023572399636304,0.9791514590957301
341,2011-04-30T00:00:00.000+0000,0.005193898,0.001856938,0.00333696,0.019674410000000003,-0.023539416,-0.02077393,0.014838050000000005,1,0,0.001856938,1.003511057104838,1.0254731101209398,0.9809696826478804
342,2011-05-31T00:00:00.000+0000,0.000834144,-0.007656539,0.008490683,0.00333696,0.019674410000000003,-0.023539416,-0.02077393,1,0,-0.007656539,1.0043481298320556,1.0176215352598474,0.9734588500148693
343,2011-06-30T00:00:00.000+0000,0.00544057,0.009008019,-0.003567449000000001,0.008490683,0.00333696,0.019674410000000003,-0.023539416,0,1,0.00544057,1.009812356136776,1.0267882893842772,0.9787550210304947
344,2011-07-31T00:00:00.000+0000,-0.050265951,-0.039695752,-0.010570199000000002,-0.003567449000000001,0.008490683,0.00333696,0.019674410000000003,0,1,-0.050265951,0.9590531777240103,0.9860291560923747,0.929556969102372
345,2011-08-31T00:00:00.000+0000,-0.068249042,-0.062892945,-0.00535609699999999,-0.010570199000000002,-0.003567449000000001,0.008490683,0.00333696,0,0,-0.062892945,0.8935987171172909,0.9240148786098606,0.8710943937702498
346,2011-09-30T00:00:00.000+0000,-0.062007234,-0.045052535,-0.016954699000000004,-0.00535609699999999,-0.010570199000000002,-0.003567449000000001,0.008490683,0,0,-0.045052535,0.8381891323628993,0.8823856659507692,0.8318493831066118
347,2011-10-31T00:00:00.000+0000,0.083166889,0.064948678,0.018218210999999998,-0.016954699000000004,-0.00535609699999999,-0.010570199000000002,-0.003567449000000001,1,0,0.064948678,0.9078987148951307,0.9396954484404212,0.8858769008345018
